In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("cleaned_data.csv")

In [5]:
df.columns.to_list()

['Age',
 'Annual_Income',
 'Monthly_Inhand_Salary',
 'Num_Bank_Accounts',
 'Num_Credit_Card',
 'Interest_Rate',
 'Num_of_Loan',
 'Delay_from_due_date',
 'Num_of_Delayed_Payment',
 'Changed_Credit_Limit',
 'Num_Credit_Inquiries',
 'Credit_Mix',
 'Outstanding_Debt',
 'Credit_Utilization_Ratio',
 'Credit_History_Age',
 'Total_EMI_per_month',
 'Amount_invested_monthly',
 'Monthly_Balance',
 'Credit_Score',
 'Credit-Builder Loan',
 'Personal Loan',
 'Debt Consolidation Loan',
 'Student Loan',
 'Payday Loan',
 'Mortgage Loan',
 'Auto Loan',
 'Home Equity Loan',
 'Month_August',
 'Month_February',
 'Month_January',
 'Month_July',
 'Month_June',
 'Month_March',
 'Month_May',
 'Occupation_Architect',
 'Occupation_Developer',
 'Occupation_Doctor',
 'Occupation_Engineer',
 'Occupation_Entrepreneur',
 'Occupation_Journalist',
 'Occupation_Lawyer',
 'Occupation_Manager',
 'Occupation_Mechanic',
 'Occupation_Media_Manager',
 'Occupation_Musician',
 'Occupation_Scientist',
 'Occupation_Teacher',
 'Oc

In [6]:
X, y = df.drop("Credit_Score", axis=1), df["Credit_Score"]

In [7]:
from imblearn.over_sampling import SMOTE
rus = SMOTE(sampling_strategy='auto')
X_data_rus, y_data_rus = rus.fit_resample(X, y)

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import PowerTransformer
scalar = PowerTransformer(method="yeo-johnson", standardize=True).fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)


Model Building

In [10]:
from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
    HistGradientBoostingClassifier
)
# from xgboost import XGBClassifier

bagging = BaggingClassifier(n_jobs=-1)
extraTrees = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
randomForest = RandomForestClassifier(n_jobs=-1)
histGradientBoosting = HistGradientBoostingClassifier()
# XGB = XGBClassifier(n_jobs=-1)

model = StackingClassifier([
    ('bagging', bagging),
    ('extraTress', extraTrees),
    ('randomforest', randomForest),
    ('histGradientBoosting', histGradientBoosting),
    # ('XGB', XGB)
], n_jobs=-1)

Model Fitting

In [11]:
model.fit(X_train, y_train)

/Users/chanit/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('bagging', BaggingClassifier(n_jobs=-1)),
                               ('extraTress',
                                ExtraTreesClassifier(max_depth=10, n_jobs=-1)),
                               ('randomforest',
                                RandomForestClassifier(n_jobs=-1)),
                               ('histGradientBoosting',
                                HistGradientBoostingClassifier())],
                   n_jobs=-1)

In [12]:
print("Train Score: ",model.score(X_train, y_train))
print("Test Score: ",model.score(X_test, y_test))

Train Score:  0.9999625
Test Score:  0.7943


In [13]:
from sklearn.svm import SVC
# Create a Linear SVC model
linear_svc = SVC(kernel='linear', C=1.0)  # You may need to tune the C parameter

# Train the model
linear_svc.fit(X_train, y_train)

# Evaluate the model
training_score = linear_svc.score(X_train, y_train)
test_score = linear_svc.score(X_test, y_test)
mean_cross_val_score = cross_val_score(linear_svc, X, y, cv=5).mean()

# Number of hyperparameters in the model
num_hyperparameters = len(linear_svc.get_params())

# Print the results
print(f"Training Score: {training_score:.4f}")
print(f"Test Score: {test_score:.4f}")
print(f"Mean Cross-Validation Score: {mean_cross_val_score:.4f}")
print(f"Number of Hyperparameters: {num_hyperparameters}")